# 第六节 LangChain 表达式语言 (LCEL)

**从链到自定义链**

**LangChain 表达式语言**，或 LCEL，是一种声明式的方法，可以轻松地将链组合在一起。 LCEL 从第一天开始设计就是为了 支持将原型直接投入生产，无需代码更改，从最简单的 “提示 + LLM” 链到最复杂的链（我们见过人们成功地在生产中运行具有数百个步骤的 LCEL 链）。以下是您可能想要使用 LCEL 的一些原因：

**流式传输支持** 当您使用 LCEL 构建链时，您将获得最佳的首次令牌时间（直到第一个输出块出现的时间）。对于某些链，这意味着例如我们直接从 LLM 流式传输令牌到流式输出解析器，您将以与 LLM 提供者输出原始令牌相同的速率获得解析后的增量输出块。

**异步支持** 使用 LCEL 构建的任何链都可以使用同步 API（例如在 Jupyter 笔记本中进行原型设计）以及异步 API（例如在 LangServe 服务器中）进行调用。这使得您可以在原型和生产中使用相同的代码，具有出色的性能，并且能够在同一个服务器上处理许多并发请求。

**优化的并行执行** 每当您的 LCEL 链具有可以并行执行的步骤（例如，如果您从多个检索器中获取文档），我们会自动执行它，无论是在同步还是异步接口中，以实现尽可能小的延迟。

**重试和回退** 为您的 LCEL 链的任何部分配置重试和回退。这是使您的链在规模上更可靠的绝佳方式。我们目前正在努力为重试/回退添加流式传输支持，以便您可以在没有任何延迟成本的情况下获得增加的可靠性。

**访问中间结果** 对于更复杂的链，通常非常有用，即使在最终输出生成之前，也能访问中间步骤的结果。这可以用于让用户知道正在发生什么，或者仅仅是为了调试您的链。您可以流式传输中间结果，它在每个 LangServe 服务器上都是可用的。

**输入和输出模式** 输入和输出模式为每个 LCEL 链提供了 Pydantic 和 JSONSchema 模式，这些模式是从您的链结构中推断出来的。这可以用于输入和输出的验证，并且是 LangServe 的一个重要组成部分。

**无缝 LangSmith 跟踪集成** 随着您的链变得越来越复杂，越来越重要的是要了解每个步骤中确切发生的事情。 使用 LCEL，所有 步骤都自动记录到 LangSmith，以实现最大的可观察性和可调试性。

**无缝 LangServe 部署集成** 使用 LCEL 创建的任何链都可以轻松地使用 LangServe 进行部署。

https://langchain114.com/docs/expression_language

https://blog.csdn.net/Attitude93/article/details/136531425

In [1]:
#导入语言模型
import os
from langchain_community.llms import Tongyi
from langchain_community.llms import SparkLLM
from langchain_community.llms import QianfanLLMEndpoint

import pandas as pd
#导入模版
from langchain.prompts import PromptTemplate

#导入聊天模型
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain_community.chat_models import ChatSparkLLM
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_community.chat_models import QianfanChatEndpoint

from langchain.chains import LLMChain

#输入三个模型各自的key

os.environ["DASHSCOPE_API_KEY"] = ""

os.environ["IFLYTEK_SPARK_APP_ID"] = ""
os.environ["IFLYTEK_SPARK_API_KEY"] = ""
os.environ["IFLYTEK_SPARK_API_SECRET"] = ""

os.environ["QIANFAN_AK"] = ""
os.environ["QIANFAN_SK"] = ""

In [2]:
model_ty = Tongyi(temperature=0.1)
model_qf = QianfanLLMEndpoint(temperature=0.1)
chat_ty = ChatTongyi()

## 一个最简单的例子：prompt + model + output parser

https://langchain114.com/docs/expression_language/interface

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser #仅仅是让输出对象成为字符串

prompt = ChatPromptTemplate.from_template("告诉我一个有关 {topic}的笑话")
model = ChatTongyi()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [4]:
chain.invoke({"topic":"绵羊"})

'当然，这是一个简短的绵羊笑话：\n\n为什么绵羊总是参加瑜伽课？ \n\n因为它们想变得更加咩力无穷（flexible）！'

In [41]:
chain.invoke({"topic":"ice cream"})

AIMessage(content="Sure, here's a short ice cream joke for you:\nWhy did the ice cream truck break down? Because it had a meltdown!")

In [49]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [50]:
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [51]:
chain.output_schema.schema()

{'title': 'StrOutputParserOutput', 'type': 'string'}

## 操作单变量输入和输出

RunnableParallel 可用于操作一个 Runnable 的输出，使其匹配序列中下一个 Runnable 的输入格式。

在这里，prompt 的输入预期是一个带有“context”和“question”键的映射。用户输入仅是问题。因此，我们需要使用检索器获取上下文，并将用户输入作为“question”键传递。

* 代码中每个 "|" 前后的元素都可看作是一个Runnable。

其中的 {"context": retriever, "question": RunnablePassthrough()} 就是RunnableParallel，它的作用就是将输入组成 context 和 question 为key的字典格式，传递给 prompt。

RunnableParallel 的使用可以有以下三种形式，三种形式等价：

* RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
* {"context": retriever,"question": RunnablePassthrough()}
* RunnableParallel(context=retriever, question=RunnablePassthrough())

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.vectorstores import Chroma
from langchain_community.embeddings import QianfanEmbeddingsEndpoint

vectorstore = Chroma.from_texts(
["小明在华为工作"], embedding=QianfanEmbeddingsEndpoint()
)
retriever = vectorstore.as_retriever()
template ="""仅根据以下上下文回答问题：
{context}

问题：{question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatTongyi()

retrieval_chain =(
{"context": retriever,"question": RunnablePassthrough()}
| prompt
| model
| StrOutputParser()
)

[INFO] [03-26 09:34:43] openapi_requestor.py:316 [t:5080]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-26 09:34:43] oauth.py:207 [t:5080]: trying to refresh access_token for ak `og6mWr***`
[INFO] [03-26 09:34:43] oauth.py:220 [t:5080]: sucessfully refresh access_token


In [4]:
retrieval_chain.invoke("小强在哪里工作？")

[INFO] [03-26 09:34:45] openapi_requestor.py:316 [t:6444]: requesting llm api endpoint: /embeddings/embedding-v1
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


'根据上下文，无法确定小强在哪里工作。'

### classwork 1

* 把下面的代码改写成 LCEL 链的形式

In [60]:
llm = Tongyi(temperature=0.3)
 
prompt = PromptTemplate(
    input_variables=["product"],
    template="我希望你能充当新公司的命名顾问。一个生产{product}的公司好的中文名字是什么,请给出五个选项？",
)

chain = LLMChain(llm=model_ty, prompt=prompt)

chain.invoke("彩色袜子")

{'product': '彩色袜子',
 'text': '当然可以，以下是五个为您的彩色袜子公司建议的中文名字：\n\n1. 彩韵袜业：这个名字融合了色彩和韵律的元素，暗示袜子的多彩和时尚感。\n2. 明彩足下：寓意穿上公司的袜子，能让人的脚步充满明亮和活力，同时也突出了彩色的特点。\n3. 七彩虹袜：七彩虹代表了所有颜色，简洁且易于记忆，让人联想到各种色彩的袜子。\n4. 色舞足尖：形象地描绘出袜子的色彩在脚尖舞动的画面，富有动态美和艺术感。\n5. 彩织悦步：强调袜子的制作精细（彩织）以及带给人们愉快的步伐体验。\n\n希望这些建议能对您有所帮助！如果您需要更多的选择或者有特定的要求，请告诉我。'}

* 把下面的代码改写成 LCEL 链的形式

In [85]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="请列出五个 {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

In [86]:
input0 = prompt.format(subject="中国的名山")
output = model_ty.invoke(input0)
res0=output_parser.parse(output)

In [87]:
res0

['泰山', '黄山', '华山', '衡山', '嵩山']

* 把下面的代码改写成 LCEL 链的形式

In [124]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

model_qf = QianfanLLMEndpoint(temperature=0.1)
loader_txt = TextLoader(r'G:\云岚宗.txt', encoding='utf8')
docs_txt = loader_txt.load()
text_splitter_txt = RecursiveCharacterTextSplitter(chunk_size = 384, chunk_overlap = 0, separators=["\n\n", "\n", " ", "", "。", "，"])
documents_txt = text_splitter_txt.split_documents(docs_txt)
embeddings_qf = QianfanEmbeddingsEndpoint()
vectordb = Chroma.from_documents(documents=documents_txt, embedding=embeddings_qf)

prompt = ChatPromptTemplate.from_template("""使用下面的语料来回答本模板最末尾的问题。如果你不知道问题的答案，直接回答 "我不知道"，禁止随意编造答案。
        为了保证答案尽可能简洁，你的回答必须不超过三句话，你的回答中不可以带有星号。
        请注意！在每次回答结束之后，你都必须接上 "感谢你的提问" 作为结束语
        以下是一对问题和答案的样例：
            请问：秦始皇的原名是什么
            秦始皇原名嬴政。感谢你的提问。
        
        以下是语料：
<context>
{context}
</context>

Question: {input}""")
#创建检索链
document_chain = create_stuff_documents_chain(model_qf, prompt)

retriever = vectordb.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

[INFO] [03-25 23:25:55] openapi_requestor.py:316 [t:32248]: requesting llm api endpoint: /embeddings/embedding-v1


In [106]:
response = retrieval_chain.invoke({
    "input": "纳兰桀是谁?"
})
print(response["answer"])

[INFO] [03-25 23:19:10] openapi_requestor.py:316 [t:5608]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-25 23:19:10] openapi_requestor.py:316 [t:31424]: requesting llm api endpoint: /chat/eb-instant


纳兰桀是加玛帝国狮心元帅纳兰桀的孙儿。


## 多变量输入输出的操作

* 使用 Python 的 itemgetter 作为速记，从映射中提取数据时与 RunnableParallel 结合使用

* 代码中每个 "|" 前后的元素都可看作是一个Runnable。

In [113]:
from operator import itemgetter
a = [1,2,3,4,5]
b = itemgetter(0)
c = itemgetter(0,1,2)
print(b(a),c(a))

1 (1, 2, 3)


In [3]:
dict0={"语文":80,"数学":90,"英语":70,"物理":92,"化学":83}
b=itemgetter("数学")
print(b(dict0))

90


In [120]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import QianfanEmbeddingsEndpoint

vectorstore = Chroma.from_texts(
["小明在华为工作"], embedding=QianfanEmbeddingsEndpoint()
)
retriever = vectorstore.as_retriever()

template ="""仅根据以下上下文回答问题：
{context}

问题：{question}

请使用以下语言回答：{language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain =(
{
"context": itemgetter("question")| retriever,
"question": itemgetter("question"),
"language": itemgetter("language"),
}
| prompt
| model_ty
| StrOutputParser()
)

[INFO] [03-25 23:23:16] openapi_requestor.py:316 [t:32248]: requesting llm api endpoint: /embeddings/embedding-v1


In [121]:
chain.invoke({"question":"小明在哪里工作","language":"英语"})

[INFO] [03-25 23:23:18] openapi_requestor.py:316 [t:10960]: requesting llm api endpoint: /embeddings/embedding-v1


'Xiaoming works at Huawei.'

## 传递数据

RunnablePassthrough 允许将输入不变地传递或添加额外的键。这通常与 RunnableParallel 结合使用，将数据分配给映射中的新键。

单独调用 RunnablePassthrough() 时，它将简单地接收输入并传递。

使用分配（RunnablePassthrough.assign(...)）调用 RunnablePassthrough 时，它将接收输入，并将添加给分配函数的额外参数。

In [72]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"]*3),
    modified=lambda x: x["num"]+1,
)

runnable.invoke({"num":1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

### classwork 2

In [67]:
prompt = PromptTemplate(
    input_variables=["product","style","num"],
    template="我希望你能充当新公司的命名顾问。公司新推出一款{product}产品，请帮给新产品取一个{style}中文名字，给出{num}个选项？",
)

chain_m = LLMChain(llm=model_ty, prompt=prompt)

In [66]:
chain_m.invoke({"product":"彩色袜子","style":"可爱","num":3})

{'product': '彩色袜子',
 'style': '可爱',
 'num': 3,
 'text': '当然可以，以下是我为您的彩色袜子产品提出的三个命名建议：\n\n1. "彩虹脚步"：这个名字寓含了袜子的多彩特性，同时也传达出活力和快乐的情绪，仿佛每一步都踩在彩虹上。\n2. "萌彩袜趣"： "萌"字代表可爱，"彩"代表颜色丰富，"袜趣"则强调产品的趣味性和个性化，适合年轻人和孩子们。\n3. "七彩踏云"： "七彩"代表多种颜色，"踏云"则给人一种轻盈、舒适的想象，适合用来形容穿着舒适，色彩丰富的袜子。\n\n希望这些建议能对您有所帮助！'}

* 改写上面的代码，用lcel实现

* 改写上面的代码，用lcel实现，但数量要在输入的基础上加一

* 改写上面《云岚宗》的rag案例，通过itemgetter实现参数的传入

### 链与链的组合

In [43]:
#https://blog.csdn.net/oHeHui1/article/details/136038434

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatTongyi()

prompt = ChatPromptTemplate.from_template("only give me the result,no other words:the result of add 20 to {expression1}")

chain_one = prompt | llm | StrOutputParser()

second_prompt = ChatPromptTemplate.from_template(
    "only give me the sum result,no other words: the result of add {expression1} to {expression2}"
)

chain_two = (
    {"expression1": chain_one, "expression2": itemgetter("expression2")}
    | second_prompt
    | llm
    | StrOutputParser()
)


In [45]:
print(chain_two.invoke({"expression1": "3","expression2": "10"}))

The sum is 23.


## 自定义函数

* RunnableLambda

In [48]:
from langchain_core.runnables import RunnableLambda


def length_function(text):
    return len(text)


def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("what is {a} + {b}")

chain1 = prompt | model_ty

chain =(
{
    "a": itemgetter("foo") | RunnableLambda(length_function),
    "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}
    | RunnableLambda(multiple_length_function),
}
| prompt
| model_ty
)

In [49]:
chain.invoke({"foo":"bar","bar":"gah"})

'3 + 9 is equal to 12.'

## 参考链接

https://blog.csdn.net/weixin_50516745/article/details/132103832

https://blog.csdn.net/qq128252/article/details/132870134

https://gitcode.csdn.net/65ed78c81a836825ed79a6d9.html

https://www.jianshu.com/p/9e646767e5b6

https://brightinventions.pl/blog/introducing-langchain-agents-tutorial-with-example/

https://api.python.langchain.com/en/latest/langchain_api_reference.html#module-langchain.agents

https://wangwei1237.github.io/LLM_in_Action/langchain_agent_react.html

* LECL

https://langchain114.com/docs/expression_language/why

https://blog.csdn.net/Attitude93/article/details/136531425


* RAG

https://cloud.tencent.com.cn/developer/article/2369731

## 